# QP 2 Analysis 
## 12-18-2025

In [1]:
# Packages

library(tidyverse)
library(easystats)

library(arrow)
library(data.table)
library(dtplyr)

library(ggplot2)
library(ggthemes)
library(patchwork)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   4.0.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
# Attaching packages: easystats 0.7.5
✔ bayestestR  0.17.0   ✔ correlation 0.8.8 
✔ datawizard  1.3.0    ✔ effectsize  1.0.1 
✔ insight     1.4.4    ✔ modelbased  0.13.1
✔ performance 0.15.3   ✔ parameters  0.28.3
✔ report      0.6.2    ✔ see         0.12.0



Attaching package: 'arrow'


The following object is masked from 'package:lubridate':

    duration


The following object is masked from 'package:utils':

    timestamp



Attaching package: 'data.table'


The foll

In [ ]:
# Read from CSV


files <- list.files(path = "D:/BNC Full Data/12-17 Run/CSV",
                    pattern = "\\.csv$",
                    full.names = TRUE)

df_full <- read_csv(files, id = "file_name") 



    # mutate(definiteness = factor(levels = c("indef", "def"))) %>% 
    # mutate(argPos = factor(levels = c("obj", "sbj")))

write_parquet(df_full, "Data/Full Data Unfactored.parquet")
###



Rows: 111571140 Columns: 41
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (19): bnc_id, consecutive_id, filename, modality, s_text, w_tok, phr_tok...
dbl (17): s_verb_cnt, s_aux_cnt, s_tot_obj_cnt, s_dir_obj_cnt, s_ind_obj_cnt...
lgl  (4): s_trans, is_noun, is_bare_np, is_np_head

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


In [3]:
summary(df_full)

ERROR: Error: object 'df_full' not found


**Data Attributes:**

- Total tokens: 
- Total sentences: 
- Transitivity: Transitive: ; Intransitive:


In [ ]:
# Turns into lazy data.table for memory and speed, then clears full df until after cleanup 
# (requires either arrow or data.table and dtplyr): 
df_full_laz <- lazy_dt(df_full) 
    rm(df_full)
    gc()

In [ ]:
        filter(
        modality == "written", 
        s_verb_count  == 1,
        s_aux_count == 0,
        s_sbj_count == 1,
        s_tot_obj_count %in% 1,
        Sent_Dir_Object_Count == 1 ,
        Sent_Ind_Object_Count == 0,
        Sent_Sub_Conj_Count == 0,
        Sent_Coord_Conj_Count == 0, 
        Clausal_Complement_Count == 0,
        Sent_Relative_Clause_Count == 0, 
        Sent_Adv_Clause_Count == 0, 
        Sent_Prep_Phrase_Count == 0,
        Sent_Comma_Count == 0,
        !str_detect(Sentence_Text, "\\?"),
        Sent_Transitive == TRUE,
    )  %>%
